## Finetuning DistilBERT using SQuAD 2.0

This notebook contains the following fine-tuning groups:

....

The starting-point for code in this file was found in the Medium blog post titled, Question Answering with DistilBERT (https://medium.com/@sabrinaherbst/question-answering-with-distilbert-ba3e178fdf3d). Main differences include:
 - The DistilBERT model was pre-trined using SQuAD 2.0, rather than SQuAD 1.0
 - Exploring traditional split that included unseen test data  (i.e. not validation data)
 - Addition of adhoc dropout rate setting and number of attention head setting
 - Addition of performance analysis for each category

....

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load libraries

from transformers import DistilBertModel, DistilBertForMaskedLM, DistilBertConfig, \
            DistilBertTokenizerFast, AutoTokenizer, BertModel, BertForMaskedLM, BertTokenizerFast, BertConfig
from torch import nn
from pathlib import Path
import torch
import pandas as pd
from typing import Optional
from tqdm.auto import tqdm
from torch.optim import AdamW, RMSprop
import numpy as np

import sys
sys.path.append('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project')
from qa_model import QuestionDistilBERT, SimpleQuestionDistilBERT, ReuseQuestionDistilBERT, Dataset, test_model
from util import eval_test_set, count_parameters, print_test_set_incorrect_predictions, \
                 analyze_test_set_performance, eval_test_set_by_category, eval_test_set_pure, display_category_examples
from my_distilbert import QADataset

# Load tokenizer


In [ ]:
# Load DistilBERT tokenizer, use uncased (lowercase) vocabulary

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Load data_2 (Traditional split)

Note: Dataset loader is the "Variable-Length Trunc" described in the report

In [ ]:
# Get paths for all SQuAD dataset text files in training directory -- data_2, Traditional Split
squad_paths_2 = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/training_squad').glob('**/*.txt')]

# Create full training dataset
dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/training_squad').glob('**/*.txt')],
                  natural_question_paths=None,
                  hotpotqa_paths=None,
                  tokenizer=tokenizer)

# Create a half size training dataset
full_size = len(dataset_2)
half_size = full_size // 2  # Integer division to get exact half
subset_indices = torch.randperm(full_size)[:half_size]
dataset_2_half = torch.utils.data.Subset(dataset_2, subset_indices)
loader_2_half = torch.utils.data.DataLoader(dataset_2_half, batch_size=8)
loader_2 = torch.utils.data.DataLoader(dataset_2, batch_size=8)

# Print sizes for verification
print(f"Original Training Dataset Size: {full_size}")
print(f"Half Training Dataset Size: {len(dataset_2_half)}")

## load the validation dataset -- used to be labeled as "test", test_dataset changed to val_dataset, test_loader changed to val_loader
val_dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/validation_squad').glob('**/*.txt')],
                       natural_question_paths=None,
                       hotpotqa_paths = None, tokenizer=tokenizer)
val_loader_2 = torch.utils.data.DataLoader(val_dataset_2, batch_size=4)
print(f"Approximate Validation Dataset Size: {len(val_dataset_2)}")

## load the test dataset -- test_dataset and test_loader should not be used during training
test_dataset_2 = Dataset(squad_paths = [str(x) for x in Path('/content/drive/MyDrive/distilBERT_SQuAD2_w266Project/data_2/test_squad').glob('**/*.txt')],
                       natural_question_paths=None,
                       hotpotqa_paths = None, tokenizer=tokenizer)
test_loader_2 = torch.utils.data.DataLoader(test_dataset_2, batch_size=4)
print(f"Approximate Test Dataset Size: {len(test_dataset_2)}")

Original Training Dataset Size: 113000
Half Training Dataset Size: 56500
Approximate Validation Dataset Size: 14000
Approximate Test Dataset Size: 14000


# Model_23
Train DistilBERT using SQuAD 2.0 --- data_2, Traditional Split
Alter training parameters to 10 epochs with dropout rate of 0.18, additional attention head added to the model (12 heads)

In [ ]:
# Load pre-trained DistilBERT model for masked language modeling
model_23 = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

# Get model configuration (architecture, hyperparameters)
config_23 = DistilBertConfig.from_pretrained("distilbert-base-uncased")

# Extract base DistilBERT model without MLM head
mod_23 = model_23.distilbert

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_23 = QuestionDistilBERT(mod_23)
model_23.set_dropout_rate(0.18)
model_23.set_num_heads(2)  # number of heads must divide evenly into the embedding dimension (768)
model_23.to(device)
print(type(mod_23))
print(mod_23)

# Verify the dropout rates for each layer
for name, module in model_23.named_modules():
    if isinstance(module, nn.Dropout):
        print(f"{name}: {module.p}")


Updated dropout rate to 0.18

Updated number of attention heads to 2
<class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'>
DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.18, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.18, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn)

In [ ]:
# Set model to training mode (enables dropout, batch norm, etc.)
model_23.train()

# Initialize RMSprop optimizer and learning rate of 9e-5
optim = AdamW(model_23.parameters(), lr=4e-5)

In [ ]:
# Train the DistilBERT model with data_2 for 3 epochs
# Validation data used to evaluate performance during training

epochs = 15

for epoch in range(epochs):
   # Training loop
   loop = tqdm(loader_2, leave=True)  # Progress bar for training batches
   model_23.train()  # Set model to training mode
   mean_training_loss = []

   for batch in loop:
       # Zero gradients at start of each batch
       optim.zero_grad()

       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass
       outputs = model_23(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Backward pass and optimization
       loss.backward()
       optim.step()

       # Track and display training progress
       mean_training_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch}')
       loop.set_postfix(loss=loss.item())
   print("Mean Training Loss", np.mean(mean_training_loss))

   # Validation loop
   loop = tqdm(val_loader_2, leave=True)  # Progress bar for validation batches
   model_23.eval()  # Set model to evaluation mode
   mean_val_loss = []

   for batch in loop:
       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass (no gradients needed for validation)
       outputs = model_23(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Track and display validation progress
       mean_val_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch} Validation set')
       loop.set_postfix(loss=loss.item())
   print("Mean Validation Loss", np.mean(mean_val_loss))

  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 3.063862338699071


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 2.4031164364814757


  0%|          | 0/14125 [00:00<?, ?it/s]

...

Mean Validation Loss 1.3838618195525239


  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 1.2911832279325586


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.4024180200078658


In [ ]:
# Save trained model parameters (weights & biases)
torch.save(model_23.state_dict(), "simple_distilbert_qa_data_23.model")

In [ ]:
# Initialize a new instance of our custom QA model
model_23 = QuestionDistilBERT(mod_23)

# Load previously saved model parameters from disk
model_23.load_state_dict(torch.load("simple_distilbert_qa_data_23.model"))

<ipython-input-12-8aaeaf6ead89>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_23.load_state_dict(torch.load("simple_distilbert_qa_data_23.model"))


<All keys matched successfully>

In [ ]:
# Evaulate data_2 performance for using test data
eval_test_set(model_23, tokenizer, test_loader_2, device)

100%|██████████| 3500/3500 [01:33<00:00, 37.30it/s]

Mean EM:  0.4097857142857143
Mean F-1:  0.44583127308825937


In [ ]:
# Evaulate data_2 performance for using test data
eval_test_set_pure(model_23, tokenizer, test_loader_2, device)

100%|██████████| 3500/3500 [01:20<00:00, 43.48it/s]

Mean Pure EM:  0.40364285714285714
Mean Pure F-1:  0.4423204025937178


In [ ]:
# Then add category evaluation
print("\nAnalyzing performance by question category:")
eval_test_set_by_category(model_23, tokenizer, test_loader_2, device)


Analyzing performance by question category:


100%|██████████| 3500/3500 [01:22<00:00, 42.68it/s]

+---------------+-------+------------+---------+---------+
| Question Type | Count | % of Total | Mean EM | Mean F1 |
+---------------+-------+------------+---------+---------+
|      what     |  8084 |   57.7%    |  0.414  |  0.454  |
|      when     |  946  |    6.8%    |  0.408  |  0.458  |
|      how      |  1400 |   10.0%    |  0.406  |  0.457  |
|      who      |  1453 |   10.4%    |  0.435  |  0.473  |
|      why      |  175  |    1.2%    |  0.354  |  0.428  |
|     other     |  278  |    2.0%    |  0.360  |  0.403  |
|     where     |  619  |    4.4%    |  0.368  |  0.410  |
|     which     |  1045 |    7.5%    |  0.351  |  0.392  |
+---------------+-------+------------+---------+---------+


{'what': {'count': 8084,
  'percentage': 57.74285714285714,
  'mean_em': 0.4140277090549233,
  'mean_f1': 0.4535576544109862},
 'when': {'count': 946,
  'percentage': 6.757142857142857,
  'mean_em': 0.4080338266384778,
  'mean_f1': 0.4583942029290492},
 'how': {'count': 1400,
  'percentage': 10.0,
  'mean_em': 0.4064285714285714,
  'mean_f1': 0.457116736904418},
 'who': {'count': 1453,
  'percentage': 10.378571428571428,
  'mean_em': 0.4349621472814866,
  'mean_f1': 0.4729136102471129},
 'why': {'count': 175,
  'percentage': 1.25,
  'mean_em': 0.35428571428571426,
  'mean_f1': 0.4277317867229193},
 'other': {'count': 278,
  'percentage': 1.9857142857142858,
  'mean_em': 0.3597122302158273,
  'mean_f1': 0.40260717337508095},
 'where': {'count': 619,
  'percentage': 4.421428571428572,
  'mean_em': 0.3683360258481422,
  'mean_f1': 0.4102229725350656},
 'which': {'count': 1045,
  'percentage': 7.464285714285715,
  'mean_em': 0.35119617224880384,
  'mean_f1': 0.39230557597383076}}

In [ ]:
# Get examples by category
examples = display_category_examples(model_23, tokenizer, test_loader_2, device)

100%|██████████| 3500/3500 [01:21<00:00, 43.03it/s]


==================== WHAT Questions ====================

✅ Successful Examples (EM=1):

1. Question: what field of study speculates about science fiction? there is considerable speculation both in science and science fiction as to why the observable universe is apparently almost entirely matter, and whether other places are almost entirely antimatter instead. in the early universe, it is thought that matter and antimatter were equally represented, and the disappearance of antimatter requires an asymmetry in physical laws called the charge parity ( or cp symmetry ) violation. cp symmetry violation can be obtained from the standard model, but at this time the apparent asymmetry of matter and antimatter in the visible universe is one of the great unsolved problems in physics. possible processes by which it came about are explored in more detail under baryogenesis.
   Predicted: 
   Truth: 

2. Question: what tradition in england, france, and portugal developed entitling certain politica

In [ ]:
# For comprehensive test set analysis of data_2
test_stats = analyze_test_set_performance(model_9, tokenizer, test_loader_2, device)
print_test_set_incorrect_predictions(model_9, tokenizer, test_loader_2, device)
analyze_test_set_statistics(model_9, tokenizer, test_loader_2, device)


Beginning Test Set Performance Analysis

Analyzing Test Set Statistics
Note: This analysis is being performed on the test set (unseen data)

Test Set Prediction Statistics:
----------------------------------------
Total test examples evaluated: 14000
Correct predictions: 6583
Test set accuracy: 47.02%
Average span length difference: 5.27 tokens
Average position difference: 27.29 tokens
Average length of incorrect answers: 4.62 words

Analyzing Test Set Incorrect Predictions
Note: This analysis is being performed on the test set (unseen data)

Found 5 incorrect test set predictions:

Test Example 1:
----------------------------------------

Context:
[CLS] what ' s another thing the paper showed hostility to? [SEP] despite its initial o[PRED_START]  [PRED_END]pposition [TRUE_START] to the c [TRUE_END]losures, until 1997, the newspaper repeatedly called for the implementation of further thatcherite policies, such as royal mail privatisation, [ verification needed ] and social security cu

{'total_examples': 14000,
 'correct_predictions': 6583,
 'accuracy': 47.02142857142857,
 'avg_span_diff': tensor(5.2689, device='cuda:0'),
 'avg_position_diff': tensor(27.2853, device='cuda:0'),
 'avg_error_length': 4.618039638667925}

# Model_10
Train DistilBERT using SQuAD 2.0 --- data_2, Traditional Split
Alter training parameters with 20 epochs and dropout rate of 0.18, additional attention head added to the model (12 heads)

Note: the model lost connectivity while training, so the it dropped data from epoch 6

In [ ]:
# Load pre-trained DistilBERT model for masked language modeling
model_10 = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased")

# Get model configuration (architecture, hyperparameters)
config_10 = DistilBertConfig.from_pretrained("distilbert-base-uncased")

# Extract base DistilBERT model without MLM head
mod_10 = model_10.distilbert

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_10 = QuestionDistilBERT(mod_10)
model_10.set_dropout_rate(0.17)
model_10.set_num_heads(12)  # number of heads must divide evenly into the embedding dimension (768)
model_10.to(device)
print(type(mod_10))
print(mod_10)

# Verify the dropout rates for each layer
for name, module in model_10.named_modules():
    if isinstance(module, nn.Dropout):
        print(f"{name}: {module.p}")


Updated dropout rate to 0.17

Updated number of attention heads to 12
<class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'>
DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.17, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): DistilBertSdpaAttention(
          (dropout): Dropout(p=0.17, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn

In [ ]:
# Set model to training mode (enables dropout, batch norm, etc.)
model_10.train()

# Initialize RMSprop optimizer and learning rate of 4e-5
optim = RMSprop(model_10.parameters(), lr=4e-5)

In [ ]:
# Train the DistilBERT model with data_2 for 3 epochs
# Validation data used to evaluate performance during training

epochs = 20

for epoch in range(epochs):
   # Training loop
   loop = tqdm(loader_2, leave=True)  # Progress bar for training batches
   model_10.train()  # Set model to training mode
   mean_training_loss = []

   for batch in loop:
       # Zero gradients at start of each batch
       optim.zero_grad()

       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass
       outputs = model_10(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Backward pass and optimization
       loss.backward()
       optim.step()

       # Track and display training progress
       mean_training_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch}')
       loop.set_postfix(loss=loss.item())
   print("Mean Training Loss", np.mean(mean_training_loss))

   # Validation loop
   loop = tqdm(val_loader_2, leave=True)  # Progress bar for validation batches
   model_10.eval()  # Set model to evaluation mode
   mean_val_loss = []

   for batch in loop:
       # Move batch data to GPU/CPU device
       input_ids = batch['input_ids'].to(device)
       attention_mask = batch['attention_mask'].to(device)
       start = batch['start_positions'].to(device)
       end = batch['end_positions'].to(device)

       # Forward pass (no gradients needed for validation)
       outputs = model_10(input_ids, attention_mask=attention_mask,
                      start_positions=start, end_positions=end)
       loss = outputs['loss']

       # Track and display validation progress
       mean_val_loss.append(loss.item())
       loop.set_description(f'Epoch {epoch} Validation set')
       loop.set_postfix(loss=loss.item())
   print("Mean Validation Loss", np.mean(mean_val_loss))

  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 3.188515219882526


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 2.622334641746112


  0%|          | 0/14125 [00:00<?, ?it/s]

...

Mean Validation Loss 1.4581585787939173


  0%|          | 0/14125 [00:00<?, ?it/s]

Mean Training Loss 1.3748717046300922


  0%|          | 0/3500 [00:00<?, ?it/s]

Mean Validation Loss 1.5410569703302213


In [ ]:
# Save trained model parameters (weights & biases)
torch.save(model_10.state_dict(), "simple_distilbert_qa_data_10.model")

In [ ]:
# Initialize a new instance of our custom QA model
model_10 = QuestionDistilBERT(mod_10)

# Load previously saved model parameters from disk
model_10.load_state_dict(torch.load("simple_distilbert_qa_data_10.model"))

<ipython-input-45-1fd6e30d85f7>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_10.load_state_dict(torch.load("simple_distilbert_qa_data_10.model"))


<All keys matched successfully>

In [ ]:
# Evaulate data_2 performance for using test data
eval_test_set(model_10, tokenizer, test_loader_2, device)

100%|██████████| 3500/3500 [01:43<00:00, 33.67it/s]

Mean EM:  0.5390714285714285
Mean F-1:  0.6004884056420002


In [ ]:
# For comprehensive test set analysis of data_2
test_stats = analyze_test_set_performance(model_10, tokenizer, test_loader_2, device)
print_test_set_incorrect_predictions(model_10, tokenizer, test_loader_2, device)
analyze_test_set_statistics(model_10, tokenizer, test_loader_2, device)


Beginning Test Set Performance Analysis

Analyzing Test Set Statistics
Note: This analysis is being performed on the test set (unseen data)

Test Set Prediction Statistics:
----------------------------------------
Total test examples evaluated: 14000
Correct predictions: 7211
Test set accuracy: 51.51%
Average span length difference: 5.25 tokens
Average position difference: 24.34 tokens
Average length of incorrect answers: 7.48 words

Analyzing Test Set Incorrect Predictions
Note: This analysis is being performed on the test set (unseen data)

Found 5 incorrect test set predictions:

Test Example 1:
----------------------------------------

Context:
[CLS] what ' s another thing the paper showed hosti[PRED_START] lity to? [SEP] despite its initial o [PRED_END]pposition [TRUE_START] to the c [TRUE_END]losures, until 1997, the newspaper repeatedly called for the implementation of further thatcherite policies, such as royal mail privatisation, [ verification needed ] and social security cu

{'total_examples': 14000,
 'correct_predictions': 7211,
 'accuracy': 51.50714285714286,
 'avg_span_diff': tensor(5.2481, device='cuda:0'),
 'avg_position_diff': tensor(24.3410, device='cuda:0'),
 'avg_error_length': 7.476064221534836}